# How to use GPU

In [14]:
import torch
import torch.nn as nn
import torchvision

torch.__version__, torchvision.__version__

('1.13.0.dev20220622', '0.14.0.dev20220622')

## Convert to CUDA tensor: cuda() -> X =>  to.('mps')

In [8]:
# x = torch.cuda.FloatTensor(2, 2)
device = torch.device('mps')

In [16]:
x = torch.FloatTensor(2, 2).to('mps')

x

/Users/cge/miniforge3/envs/pytorch/lib/python3.9/site-packages/torch/_tensor_str.py:103: UserWarning: The operator 'aten::bitwise_and.Tensor_out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  nonzero_finite_vals = torch.masked_select(tensor_view, torch.isfinite(tensor_view) & tensor_view.ne(0))


tensor([[2.7883e-37, 3.4438e-41],
        [0.0000e+00, 0.0000e+00]], device='mps:0')

In [17]:
torch.backends.mps.is_available()

True

In [18]:
# copy
# x.cuda(device=d)

In [19]:
x.device

device(type='mps', index=0)

## Convert to mps tensor: to()

In [21]:
x.to('mps')

tensor([[2.7883e-37, 3.4438e-41],
        [0.0000e+00, 0.0000e+00]], device='mps:0')

## Convert to mps tensor from CUDA tensor

In [23]:
x = torch.FloatTensor(2, 2).to('mps')

In [24]:
x = x.cpu()

In [25]:
d = torch.device('cpu')
x = x.to(d)

## Move model from CPU to GPU.

In [26]:
def print_params(model):
    for p in model.parameters():
        print(p)

In [27]:
linear = nn.Linear(2, 2)

print_params(linear)

Parameter containing:
tensor([[ 0.1012,  0.4742],
        [ 0.2245, -0.2612]], requires_grad=True)
Parameter containing:
tensor([ 0.3060, -0.6302], requires_grad=True)


In [28]:
linear = linear.to('mps')

print_params(linear)

Parameter containing:
tensor([[ 0.1012,  0.4742],
        [ 0.2245, -0.2612]], device='mps:0', requires_grad=True)
Parameter containing:
tensor([ 0.3060, -0.6302], device='mps:0', requires_grad=True)


In [29]:
linear = linear.cpu()

print_params(linear)

Parameter containing:
tensor([[ 0.1012,  0.4742],
        [ 0.2245, -0.2612]], requires_grad=True)
Parameter containing:
tensor([ 0.3060, -0.6302], requires_grad=True)


In [16]:
d = torch.device('cuda:0')
linear = linear.to(d)

print_params(linear)

Parameter containing:
tensor([[ 0.5782,  0.4288],
        [-0.0220, -0.1614]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.6881, 0.0907], device='cuda:0', requires_grad=True)


Note that nn.Module class does not have 'device' property.

In [17]:
# 모델의 경우는 각 feature마다 device가 달라 따로 상속되어 있지 않음! 
## 알고싶다면 p = next(model.parameters())
## p.device 사용! (!같은 디바이스라는 전제조건하에!)

linear.device

AttributeError: 'Linear' object has no attribute 'device'

## Tricks

In [30]:
x = torch.FloatTensor(2, 2).to('mps')

In [31]:
x.new(2, 2)

tensor([[ 0.3060, -0.6302],
        [ 0.0000,  0.0000]], device='mps:0')

In [32]:
# x랑 같은 디바이스에 같은 사이즈인 0으로 가득찬 tensor
torch.zeros_like(x)

tensor([[0., 0.],
        [0., 0.]], device='mps:0')

In [33]:
torch.ones_like(x)

tensor([[1., 1.],
        [1., 1.]], device='mps:0')

In [34]:
list(linear.parameters())[0].new(2, 2)

tensor([[-4.9366e+20,  1.1213e-36],
        [ 0.0000e+00,  8.5938e-39]])

### 결론! 서로 다른 디바이스인 tensor는 연산 불가능!